### 推荐系统整体架构图
<img src="./imgs/img18.png" width = "350" height = "350" />

内容端：新音乐会存到内容源（音乐库），内容源经过内容分析，得到结构化的内容库和内容模型，也即内容画像。

用户端：用户看到推荐列表后，会产生用户行为数据，结合物品画像，经过用户分析得到用户画像。

推荐算法：

    1.基于用户的协同过滤
        1.1寻找和你相似的用户，推荐你没听过，但他喜欢/收藏/的歌单
        
    2.基于物品的协同过滤
        2.1 构建用户歌曲评价矩阵，根据不同的用户行为赋予不同的权重。
<img src="./imgs/img19.png" width = "350" height = "350" />
<img src="./imgs/img20.png" width = "350" height = "350" />
        
        2.2 根据余弦相似度计算其他歌曲和当前歌曲的相似度。
    
    3.基于tags的推荐
        利用标签来推荐与该歌曲相似的歌曲，利用标签来推荐与该歌单相似的歌单。
        根据你对歌单的播放倾向性，收藏歌单倾向性，不难获得你对特定几个tag的倾向性，这种倾向性是用户画像的基础。
        3.1 歌单tags来源
            3.1.1 人工编辑歌单标签
            3.1.2 每首歌背后有tags，每个歌单也有tags，根据歌单的播放数、收藏数、分享数也可以决定其权威性，权威高的歌单tags，可得到每首歌的tags。
        3.2 歌曲tags来源
            3.2.1 歌手、曲风、年代、标题、时长、歌曲播放量、榜单、评论内容等。
            3.2.2 包含音乐所属专辑的tag，歌手的tag也一并继承使用。

    4.基于内容的推荐
        4.1 直接计算相似性。用户画像表示成稀疏向量，内容端也有稀疏向量，两者直接计算余弦相似度，根据相似度对推荐物品排序。
        4.2 训练预训练模型。收集点击的行为日志数据，转化为训练样本，训练预估模型。输入包括用户画像、物品的结构化内容以及上下文场景信息，如时间、位置、设备等，然后用LR或者GBDT或二者结合做训练，推荐时匹配用户行为发生的概率，按照概率排序。（这里难道是将用户和召回到的物品用spark做个拼接，然后作为模型输入？）
    
    5.采用一些关联规则做推荐
        5.1 实时情况发生的条件
            5.1.1 用户收藏了一个歌单
            5.1.2 用户收藏了一个单曲到我喜欢的音乐
            5.1.3 用户搜索了一个单曲，并且听了2遍以上
            5.1.4 用户搜索并点击了一个艺人
        5.2 艺人与歌单的关联规则
            计算喜欢这个艺人的人，也收藏了哪些歌单，当用户搜索并点击了艺人，推荐和这个艺人相关的歌单
        5.3 歌单与歌单的关联规则
            5.3.1 基于物品相似性的协同过滤算法
                根据item-cf算法得到相似性排序后的候选歌单，把用户点过或曝光过2次以上的歌单过滤掉，把相似性低于阈值的也过滤掉进行推荐。
        5.4 单曲和歌单的关联规则
            5.4.1 计算喜欢这个单曲的人，也收藏了哪些歌单，当用户收藏一个单曲后，推荐和单曲同时被收藏的歌单            
        5.5 在系统中采用多种推荐算法进行混合推荐，有些关联规则通过sql语句即可实现，有些则需要协同过滤算法配合实现
        
#### 1. 内容分析
新物品进入，需要实时进行推荐，这时候需要对内容实时分析，通过内容分析模型提取结构化信息，然后与用户画像做匹配。

内容分析模型包括：

    1）分类器模型
    2）主题模型
    3）实体识别模型
    4）嵌入模型


#### 2. 构建用户画像的方法
用户的人口统计学信息，更多是从文本提取出用户画像所需要的特征。

    1. 用户端 
        注册信息等人口统计学信息
        对歌曲的搜索（精确模型），播放（次数/时长），喜欢（喜欢时长/是否取消），收藏，评论，分享，转发
        1.1 短期特征
            点过哪些歌曲、歌曲类别、点赞、评论、收藏等，站内行为线上存200个。
        1.2 长期特征
            点击过什么类目，各类目下点了多少歌，收藏的歌单类型，收藏歌单类型个数，多长时间登录APP次数，最高频的搜索词等
    2. 物品端
        歌单：标签、描述、标题、评论等
        歌曲信息：歌名、演唱者、作曲家、填词人、年代、语言、乐器、类型、时长，风格。
        歌曲内容：主题、情感标签、歌词字段等。
    
    3. 如何从文本中提取用户画像所需特征？
        3.1 将非结构化的文本结构化，可以用到的方法有：
            3.1.1 关键词提取
            3.1.2 实体识别
            3.1.3 内容分类（专家分类）
            3.1.4 类簇编号
            3.1.5 主题模型
            3.1.6 嵌入向量
        3.2 把物品结构化数据与用户结构化数据合并
            3.2.1 把用户产生过行为的物品标签累积在一起
            3.2.2 把用户对物品的行为，有消费没消费看做一个分类问题，此时挑选感兴趣标签看做特征选择问题
        

#### 3. 冷启动的方法

1.基于热度进行推荐，推荐流行热点音乐。当用户行为达到一定程序，系统通过内容和连接进行推荐，按照一定比例推荐给用户。

2.基于内容推荐，人工标记歌曲标签，计算物品结构化信息和用户画像的相似度。

3.使用微博账号登录，从微博信息中生成个性化初始歌单（官方编辑），初期口碑不错，降低了用户成本。

4.在推荐内容上做筛选，找出优质内容，保证推荐库的健康。（官方编辑）

#### 4. 常见问题

4.1 一直推同一个歌手的歌，怎么办？

    简单的多样性过滤，限制来自同一个歌手的推荐数量

4.2 热门歌曲影响歌曲之间的相似度？ 
    
    计算相似度的时候过滤掉过于热门的歌曲
    
4.3 如何挖掘冷门歌曲
    
    基于内容的推荐系统，根据用户画像和物品的结构化数据的内容相似性做推荐
    用归一化小技巧微调一下，归一化可解决过于热门的问题